In [ ]:
!pip install unsloth transformers datasets trl --q

In [ ]:
import json
from datasets import Dataset
from unsloth import to_sharegpt

# Load and process JSONL file
data = []
with open('Data2.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line.strip())
        # Standardize key to 'input'
        if 'instruction' in item:
            item['input'] = item.pop('instruction')
        data.append(item)

# Create HuggingFace dataset
dataset = Dataset.from_list(data)

# Convert to ShareGPT format for unsloth
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{input}",
    output_column_name = "output",
    conversation_extension = 3,
)

In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.2,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
chat_template = """Given the instruction think step by step and provide a response. if the instruction is ambigous or needs more clarification ask for more information using the <ask> and </ask> tokens.
By generating <ask>......</ask>

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        num_train_epochs = 4,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 20,  # REMOVE or set to -1
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained_gguf("model", tokenizer)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)
print(tokenizer._ollama_modelfile)

In [ ]:
!ollama create deepseek_finetuned_model -f ./model/Modelfile

In [ ]:
!pip install ollama

In [ ]:
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
            messages=[{ "role": "user", "content": "I want to buy a new smartphone"
            },
                      ])

print(response.message.content)

In [ ]:
from IPython.display import Markdown
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
                       messages=[{"role": "user",
                                  "content": "How to add chart to a document?"},

                      ])

Markdown(response.message.content)

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the model directory
model_dir = './model'

# Define the destination path in Google Drive
# You can change 'my_models' to your desired folder name in Google Drive
drive_path = '/content/drive/MyDrive/my_models'

# Create the destination directory in Google Drive if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Copy the model directory to Google Drive
!cp -r {model_dir} {drive_path}

print(f"Model directory '{model_dir}' saved to '{drive_path}' in Google Drive.")